In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import random
random.seed(0)

#Additional libs
import matplotlib.patches as patches


In [ ]:
import os
import tensorflow
from keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import tensorflow_addons as tfa

In [ ]:
# Useful Configuration/Setting

# suppress display of warnings
#warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images')
test_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Test Images')

BATCH_SIZE = 32 
IMG_SIZE = 224

In [ ]:
import pandas as pd

#Lets read the Train Annotations

#Lets give our own column names as excel sheet does not have proper column names
columnNameList=['file_name','cor1','cor2','cor3','cor4','Image_Class']
trainAnno= pd.read_csv('/content/drive/MyDrive/Capstone/Annotations/Train Annotations.csv',names=columnNameList,header=0)
trainAnno.head(5)

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
files = train_data_dir.rglob('*.jpg')
input=pd.DataFrame(files,columns =['a'])
input['a']=input['a'].astype(str)
df_train=input['a'].str.split('/',expand=True)
train=input.join(df_train)[['a',8]]
train.columns=('file_path','label')

In [ ]:
train.head()
len(train)

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


8144

In [ ]:
train.head()

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


In [ ]:
train['file_name']=train['file_path'].apply(lambda getFileName:getFileName.split('/')[9])

In [ ]:
train.head()

,file_path,label,file_name
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg


In [ ]:
trainAnno.head()

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
# Lets add coordinate information to the above table
trainDF=pd.merge(train,trainAnno,on='file_name',how='inner')

In [ ]:
trainDF.head(5)

,file_path,label,file_name,cor1,cor2,cor3,cor4,Image_Class
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg,80,91,589,429,184
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg,39,35,591,413,184
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg,64,347,629,577,184
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg,118,93,380,317,184
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg,9,11,824,489,184


In [ ]:
from sklearn.model_selection import train_test_split
train_df,valid_df= train_test_split(trainDF, test_size=0.20, random_state=42, stratify=train['label'])

In [ ]:
train_df.shape
valid_df.shape

(6515, 8)

(1629, 8)

In [ ]:
#!pip install -qqq tensorflow_addons
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
!pip install -U albumentations

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-mdnjenza
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-mdnjenza
     |████████████████████████████████| 60.5 MB 1.1 MB/s 
     |████████████████████████████████| 631 kB 57.4 MB/s 
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29638 sha256=c41d402a4d9b15bc784bcb47d7ef416a7e643def2f537d07bb40c4ec44b92c02
  Stored in directory: /tmp/pip-ephem-wheel-cache-fqksfu6t/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=922857e60f75ebd75208eba474bd67cae14d182d352458e7fc33de23897f18b1
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built ImageDataAugmentor imgaug
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.

In [ ]:
# Setting the correct order of labels to prevent mismatches
class_label_list=pd.read_csv('/content/drive/MyDrive/Capstone/Car _names_and_make.txt',names=['make'])

In [ ]:
type(class_label_list)

pandas.core.frame.DataFrame

In [ ]:
class_label_list.head()

,make
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012


In [ ]:
all_classes=list(class_label_list['make'])

In [ ]:
len(all_classes)

196

In [ ]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

    
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(p=0.5,limit=25),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=1.5, contrast_limit=0.1)
    ],p=1),
    #albumentations.HueSaturationValue(p=0.5),
    #albumentations.Affine(shear=20),
    albumentations.Cutout(num_holes=1, max_h_size=16,max_w_size = 16,p=1),
    #albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=15, p=.75),
    #albumentations.Blur(blur_limit=3),
    #albumentations.OpticalDistortion()
    #albumentations.RGBShift(p=0.5),
])

# dataloaders
train_img_datagen = ImageDataAugmentor(
        augment=AUGMENTATIONS,
        #validation_split=0.2,
        seed=42
        )


val_datagen = ImageDataAugmentor()#validation_split=0.2)


train_ds = train_img_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes

        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


Found 6482 validated image filenames belonging to 196 classes.
Found 1621 validated image filenames belonging to 196 classes.


In [ ]:
(x,y)=train_ds.next()

In [ ]:
x.shape

(32, 224, 224, 3)

In [ ]:
y.shape

(32, 196)

In [ ]:
train_ds.class_indices

{'AM General Hummer SUV 2000': 0,
 'Acura Integra Type R 2001': 1,
 'Acura RL Sedan 2012': 2,
 'Acura TL Sedan 2012': 3,
 'Acura TL Type-S 2008': 4,
 'Acura TSX Sedan 2012': 5,
 'Acura ZDX Hatchback 2012': 6,
 'Aston Martin V8 Vantage Convertible 2012': 7,
 'Aston Martin V8 Vantage Coupe 2012': 8,
 'Aston Martin Virage Convertible 2012': 9,
 'Aston Martin Virage Coupe 2012': 10,
 'Audi 100 Sedan 1994': 11,
 'Audi 100 Wagon 1994': 12,
 'Audi A5 Coupe 2012': 13,
 'Audi R8 Coupe 2012': 14,
 'Audi RS 4 Convertible 2008': 15,
 'Audi S4 Sedan 2007': 16,
 'Audi S4 Sedan 2012': 17,
 'Audi S5 Convertible 2012': 18,
 'Audi S5 Coupe 2012': 19,
 'Audi S6 Sedan 2011': 20,
 'Audi TT Hatchback 2011': 21,
 'Audi TT RS Coupe 2012': 22,
 'Audi TTS Coupe 2012': 23,
 'Audi V8 Sedan 1994': 24,
 'BMW 1 Series Convertible 2012': 25,
 'BMW 1 Series Coupe 2012': 26,
 'BMW 3 Series Sedan 2012': 27,
 'BMW 3 Series Wagon 2012': 28,
 'BMW 6 Series Convertible 2007': 29,
 'BMW ActiveHybrid 5 Sedan 2012': 30,
 'BMW 

In [ ]:
type(train_ds)

ImageDataAugmentor.dataframe_iterator.DataFrameIterator

In [ ]:
# Load prepared data 


X_Train_MobileNet=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Train_MobileNet.npy")

X_Test_MobileNet=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/X_Test_MobileNet.npy")

Y_Train_MobileNet_BoxInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_BoxInput.npy")

Y_Train_MobileNet_ClassInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Train_MobileNet_ClassInput.npy")

Y_Test_MobileNet_BoxInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_BoxInput.npy")

Y_Test_MobileNet_ClassInput=np.load("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Test_MobileNet_ClassInput.npy")


In [ ]:
from tensorflow.keras import layers

def build_model(num_classes=196,IMG_SIZE=224):
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, input_tensor=inputs, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  classPredicModel = layers.Dense(num_classes, activation="softmax", name="class_output")(x)

  # Add new top layer for Regression Head
  boxPredicModel = (model.output)
  boxPredicModel = Flatten()(boxPredicModel)
  boxPredicModel = Dense(units=4, name="box_output")(boxPredicModel) # These are the 4 predicted coordinates of one BBox


  # Compile
  #return tf.keras.Model(inputs, outputs, name="EfficientNet")
  return Model(inputs=inputs, outputs=[boxPredicModel,classPredicModel])

In [ ]:
# Define evaluation metric

def IOU(y_true, y_pred):
    intersections = 0
    unions = 0
    gt = y_true
    pred = y_pred
    # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height

    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection
    # rk - again to repeat we know why he has minues , taken 2 times 

    # Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
      if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
        intersections += intersection[j]
        unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + tensorflow.keras.backend.epsilon()), 4)# rounded to 4 
    # This must match the type used in py_func
    iou = iou.astype(np.float32)
    return iou
    # this is one number for the whole model

In [ ]:
from functools import partial
INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )


optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)

model = build_model(num_classes=196,IMG_SIZE=224) 
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
epochs = 5 


hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

43950080/43941136 [==============================] - 1s 0us/step
Epoch 1/5
203/203 [==============================] - 4286s 21s/step - loss: 7.0366 - accuracy: 0.0455 - val_loss: 3.9867 - val_accuracy: 0.1659
Epoch 2/5
203/203 [==============================] - 94s 463ms/step - loss: 6.5954 - accuracy: 0.0899 - val_loss: 3.5208 - val_accuracy: 0.1925
Epoch 3/5
203/203 [==============================] - 94s 461ms/step - loss: 4.3707 - accuracy: 0.1387 - val_loss: 2.9400 - val_accuracy: 0.3023
Epoch 4/5
203/203 [==============================] - 93s 458ms/step - loss: 3.8796 - accuracy: 0.1948 - val_loss: 2.6402 - val_accuracy: 0.3732
Epoch 5/5
203/203 [==============================] - 94s 462ms/step - loss: 3.6949 - accuracy: 0.2225 - val_loss: 2.6061 - val_accuracy: 0.3689


In [ ]:
#STOP

In [ ]:
checkpoint_path = "./cp-{iter:05d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.save_weights(checkpoint_path)

In [ ]:
model.evaluate(val_ds)

51/51 [==============================] - 16s 305ms/step - loss: 2.6365 - accuracy: 0.3590


[2.6364901065826416, 0.3590376377105713]

In [ ]:
IMG_SIZE = 300
#IMG_SIZE = 128

train_ds = train_img_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

model = build_model(num_classes=196,IMG_SIZE=IMG_SIZE)
model.load_weights(checkpoint_path)

optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Found 6482 validated image filenames belonging to 196 classes.
Found 1621 validated image filenames belonging to 196 classes.


Epoch 1/5
203/203 [==============================] - 107s 476ms/step - loss: 3.6363 - accuracy: 0.2248 - val_loss: 2.4838 - val_accuracy: 0.3794
Epoch 2/5
203/203 [==============================] - 93s 460ms/step - loss: 3.6114 - accuracy: 0.2325 - val_loss: 2.2868 - val_accuracy: 0.4300
Epoch 3/5
203/203 [==============================] - 93s 460ms/step - loss: 3.3603 - accuracy: 0.2757 - val_loss: 2.1697 - val_accuracy: 0.4695
Epoch 4/5
203/203 [==============================] - 94s 462ms/step - loss: 3.2649 - accuracy: 0.2988 - val_loss: 2.1456 - val_accuracy: 0.4744
Epoch 5/5
203/203 [==============================] - 93s 460ms/step - loss: 3.2880 - accuracy: 0.2959 - val_loss: 2.1441 - val_accuracy: 0.4565


In [ ]:
checkpoint_path = "./cp-{iter:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.save_weights(checkpoint_path)

In [ ]:
for layer in model.layers[-20:]:
  if not isinstance(layer, layers.BatchNormalization):
    layer.trainable = True

INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )

model.load_weights(checkpoint_path)

model.compile(optimizer=tfa.optimizers.LAMB(), loss="categorical_crossentropy", metrics=["accuracy"])


epochs = 5  
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/5
203/203 [==============================] - 106s 470ms/step - loss: 3.1655 - accuracy: 0.3206 - val_loss: 1.9325 - val_accuracy: 0.5009
Epoch 2/5
203/203 [==============================] - 93s 455ms/step - loss: 2.9964 - accuracy: 0.3521 - val_loss: 1.8029 - val_accuracy: 0.5219
Epoch 3/5
203/203 [==============================] - 92s 456ms/step - loss: 2.8416 - accuracy: 0.3750 - val_loss: 1.6791 - val_accuracy: 0.5534
Epoch 4/5
203/203 [==============================] - 93s 459ms/step - loss: 2.7953 - accuracy: 0.3940 - val_loss: 1.6289 - val_accuracy: 0.5688
Epoch 5/5
203/203 [==============================] - 93s 456ms/step - loss: 2.6786 - accuracy: 0.4138 - val_loss: 1.5351 - val_accuracy: 0.5867


In [ ]:
#model.save_weights(checkpoint_path.format(iter=3))

In [ ]:
epochs = 30
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/30
203/203 [==============================] - 92s 455ms/step - loss: 2.6265 - accuracy: 0.4367 - val_loss: 1.4927 - val_accuracy: 0.5978
Epoch 2/30
203/203 [==============================] - 92s 455ms/step - loss: 2.5434 - accuracy: 0.4482 - val_loss: 1.4620 - val_accuracy: 0.6070
Epoch 3/30
203/203 [==============================] - 92s 453ms/step - loss: 2.4664 - accuracy: 0.4639 - val_loss: 1.3986 - val_accuracy: 0.6157
Epoch 4/30
203/203 [==============================] - 92s 453ms/step - loss: 2.4572 - accuracy: 0.4776 - val_loss: 1.3541 - val_accuracy: 0.6218
Epoch 5/30
203/203 [==============================] - 92s 452ms/step - loss: 2.4075 - accuracy: 0.4853 - val_loss: 1.3521 - val_accuracy: 0.6360
Epoch 6/30
203/203 [==============================] - 91s 450ms/step - loss: 2.3230 - accuracy: 0.5060 - val_loss: 1.2958 - val_accuracy: 0.6447
Epoch 7/30
203/203 [==============================] - 92s 451ms/step - loss: 2.2488 - accuracy: 0.5148 - val_loss: 1.2717 - val_ac

KeyboardInterrupt: ignored

In [ ]:
#model.load_weights(checkpoint_path)

In [ ]:
checkpoint_path = "./cp-{iter:04d}.ckpt"
#checkpoint_dir = os.path.dirname()
model.save_weights(checkpoint_path.format(iter=2))

In [ ]:
!ls

checkpoint				cp-{iter:26d}.ckpt.data-00000-of-00001
cp-0002.ckpt.data-00000-of-00001	cp-{iter:26d}.ckpt.index
cp-0002.ckpt.index			drive
cp-{iter:05d}.ckpt.data-00000-of-00001	sample_data
cp-{iter:05d}.ckpt.index


In [ ]:
model.load_weights(checkpoint_path.format(iter=2))

In [ ]:
model.evaluate(val_ds)

51/51 [==============================] - 17s 321ms/step - loss: 1.0948 - accuracy: 0.6915


[1.094820499420166, 0.6915484070777893]

In [ ]:
model.save_weights('/content/drive/MyDrive/Capstone_Final/PickledData/classification_best_weights.h5')

In [ ]:
model.save('/content/drive/MyDrive/Capstone_Final/PickledData/ClassficationModel.h5')

In [ ]:
### Results with 128* 128 , many epochs -> only reached .37

In [ ]:
epochs = 3
model.fit(X_Train_MobileNet,Y_Train_MobileNet_ClassInput, epochs=epochs, validation_data=(X_Test_MobileNet,Y_Test_MobileNet_ClassInput), verbose=1)

Epoch 1/3
41/41 [==============================] - 28s 549ms/step - loss: 5.9225 - accuracy: 0.0023 - val_loss: 1.1420 - val_accuracy: 0.6866
Epoch 2/3
41/41 [==============================] - 22s 544ms/step - loss: 5.7752 - accuracy: 0.0069 - val_loss: 1.1939 - val_accuracy: 0.6841
Epoch 3/3
41/41 [==============================] - 22s 551ms/step - loss: 5.6993 - accuracy: 0.0038 - val_loss: 1.3101 - val_accuracy: 0.6841
